In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 读取数据
X_all = pd.read_csv('X_filtered.csv')
Y = pd.read_csv('Y.csv')

X_train, X_test, y_train, y_test = train_test_split(X_all, Y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.ravel(), dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.ravel(), dtype=torch.long)

class NeuralNet(nn.Module):
    def __init__(self, input_dim, hidden_size1=64, hidden_size2=32, dropout_rate=0.2):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size1)
        self.bn1 = nn.BatchNorm1d(hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.bn2 = nn.BatchNorm1d(hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, 3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

def train_and_evaluate(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, 
                      learning_rate, batch_size, epochs=300, patience=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    best_train_loss = float('inf')
    no_improve_count = 0
    final_metrics = {}
    
    for epoch in range(epochs):
        model.train()
        dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        epoch_loss = 0
        batch_count = 0
        
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            batch_count += 1
        
        avg_epoch_loss = epoch_loss / batch_count
        
        # 记录最佳训练loss
        if avg_epoch_loss < best_train_loss:
            best_train_loss = avg_epoch_loss
            no_improve_count = 0
            
            # 在最佳loss时计算训练集和测试集的准确率
            model.eval()
            with torch.no_grad():
                # 训练集准确率
                train_outputs = model(X_train_tensor)
                train_predicted = torch.argmax(train_outputs, axis=1)
                train_accuracy = (train_predicted == y_train_tensor).sum().item() / len(y_train_tensor)
                
                # 测试集准确率
                test_outputs = model(X_test_tensor)
                test_predicted = torch.argmax(test_outputs, axis=1)
                test_accuracy = (test_predicted == y_test_tensor).sum().item() / len(y_test_tensor)
                
                final_metrics = {
                    'best_train_loss': best_train_loss,
                    'train_accuracy': train_accuracy,
                    'test_accuracy': test_accuracy,
                    'best_epoch': epoch
                }
        else:
            no_improve_count += 1
        
        if no_improve_count >= patience:
            print(f"Early stopping at epoch {epoch}")
            break
    
    return final_metrics

# 定义超参数网格
param_grid = {
    'learning_rate': [0.0001, 0.001, 0.01],
    'batch_size': [32, 64, 128],
    'hidden_size1': [32, 64, 128],
    'hidden_size2': [16, 32, 64],
    'dropout_rate': [0.1, 0.2, 0.3]
}

# 网格搜索
best_train_loss = float('inf')
best_params = {}
results = []

# 创建结果文件
with open('grid_search_results.txt', 'w') as f:
    f.write("Grid Search Results:\n")

for lr in param_grid['learning_rate']:
    for bs in param_grid['batch_size']:
        for hs1 in param_grid['hidden_size1']:
            for hs2 in param_grid['hidden_size2']:
                for dr in param_grid['dropout_rate']:
                    print(f"\nTesting parameters: lr={lr}, bs={bs}, h1={hs1}, h2={hs2}, dr={dr}")
                    
                    # 创建模型
                    model = NeuralNet(input_dim=X_train.shape[1], 
                                    hidden_size1=hs1,
                                    hidden_size2=hs2,
                                    dropout_rate=dr)
                    
                    # 训练和评估
                    metrics = train_and_evaluate(model, X_train_tensor, y_train_tensor, 
                                              X_test_tensor, y_test_tensor,
                                              learning_rate=lr, batch_size=bs)
                    
                    # 保存结果
                    result = {
                        'learning_rate': lr,
                        'batch_size': bs,
                        'hidden_size1': hs1,
                        'hidden_size2': hs2,
                        'dropout_rate': dr,
                        **metrics
                    }
                    results.append(result)
                    
                    # 基于训练loss选择最佳参数
                    if metrics['best_train_loss'] < best_train_loss:
                        best_train_loss = metrics['best_train_loss']
                        best_params = {
                            'learning_rate': lr,
                            'batch_size': bs,
                            'hidden_size1': hs1,
                            'hidden_size2': hs2,
                            'dropout_rate': dr
                        }
                    
                    # 打印和保存当前结果
                    result_str = (f"Parameters: lr={lr}, bs={bs}, h1={hs1}, h2={hs2}, dr={dr}\n"
                                f"Train Loss: {metrics['best_train_loss']:.4f}\n"
                                f"Train Accuracy: {metrics['train_accuracy']:.4f}\n"
                                f"Test Accuracy: {metrics['test_accuracy']:.4f}\n"
                                f"Best Epoch: {metrics['best_epoch']}\n"
                                f"{'='*50}\n")
                    
                    print(result_str)
                    with open('grid_search_results.txt', 'a') as f:
                        f.write(result_str)

# 打印最佳参数
print("\nBest parameters (based on training loss):")
for param, value in best_params.items():
    print(f"{param}: {value}")
print(f"Best training loss: {best_train_loss:.4f}")

# 将结果保存到DataFrame中
results_df = pd.DataFrame(results)
results_df.to_csv('grid_search_results.csv', index=False)

# 分析训练集准确率和测试集准确率的关系
results_df.plot.scatter(x='train_accuracy', y='test_accuracy')
plt.title('Train vs Test Accuracy')
plt.savefig('accuracy_comparison.png')
plt.close()

# 可视化不同参数对训练loss的影响
for param in ['learning_rate', 'batch_size', 'hidden_size1', 'hidden_size2', 'dropout_rate']:
    plt.figure(figsize=(10, 6))
    plt.scatter(results_df[param], results_df['best_train_loss'])
    plt.xlabel(param)
    plt.ylabel('Best Training Loss')
    plt.title(f'Impact of {param} on Training Loss')
    plt.savefig(f'{param}_vs_loss.png')
    plt.close()

Epoch 0: Loss = 1.0995
Epoch 10: Loss = 0.9021
Epoch 20: Loss = 0.8610
Epoch 30: Loss = 0.8483
Epoch 40: Loss = 0.8421
Epoch 50: Loss = 0.8362
Epoch 60: Loss = 0.8295
Early stopping at epoch 69 as loss hasn't improved for 10 epochs
Test Accuracy: 0.6350
